In [50]:
import torch
import mlx.core as mx

In [54]:
from proteus.utils import coerce_torch_to_mx

l = 2
tq = torch.randn((1, 1, l, l))
tk, tv = torch.randn_like(tq), torch.randn_like(tq)

mq, mk, mv = coerce_torch_to_mx(tq), coerce_torch_to_mx(tk), coerce_torch_to_mx(tv)

tq, tk, tv

coercing torch to mlx
coercing torch to mlx
coercing torch to mlx


(tensor([[[[-1.528901, -0.738194],
           [-0.186240,  1.351407]]]]),
 tensor([[[[1.096969, 1.236760],
           [0.830869, 1.700877]]]]),
 tensor([[[[-1.562751, -1.337172],
           [-0.880467,  1.043339]]]]))

In [55]:
torch_mask = torch.ones((l, l), dtype=torch.float32).tril()
# torch_mask = torch.zeros_like(torch_mask, dtype=torch.float32).masked_fill(torch_mask.logical_not(), float('-inf'))

mlx_mask = coerce_torch_to_mx(torch_mask)

torch_mask, mlx_mask

coercing torch to mlx


(tensor([[1., 0.],
         [1., 1.]]),
 array([[1, 0],
        [1, 1]], dtype=float32))

In [60]:
from math import sqrt
torch.set_printoptions(precision=6)

scale = sqrt(tq.size(-1))**-1

# torch_attn = torch.ops.aten.scaled_dot_product_attention.default
torch_attn = torch.ops.aten._scaled_dot_product_flash_attention_for_cpu.default
mlx_attn = mx.fast.scaled_dot_product_attention


(
    torch_attn(tq, tk, tv, attn_mask=torch_mask, scale=scale), 
    mlx_attn(mq, mk, mv, mask=mlx_mask, scale=scale)
)

((tensor([[[[-1.373101, -0.675474],
            [-1.141506,  0.132568]]]]),
  tensor([[[-0.505814,  2.998151]]])),
 array([[[[-1.3731, -0.675474],
          [-1.14151, 0.132568]]]], dtype=float32))

when torch sdpa gets a boolean tensor mask, it fills mask with -inf where the tensor is false and 0 where true, whereas MLX fills with 1 where true and 0 where false, so the solution is convert the boolean tensor manually to the -inf tensor for MLX

In [49]:
x = mx.ones((4, 4))
mask = mx.tril(mx.ones((4, 4), dtype=mx.bool_))
mx.where(mx.logical_not(mask), -float('inf'), 0.)

mx.triu(mx.full((4, 4), -float('inf')), k=1)

array([[0, -inf, -inf, -inf],
       [0, 0, -inf, -inf],
       [0, 0, 0, -inf],
       [0, 0, 0, 0]], dtype=float32)

In [53]:
torch_attn()

RuntimeError: aten::scaled_dot_product_attention() is missing value for argument 'query'. Declaration: aten::scaled_dot_product_attention(Tensor query, Tensor key, Tensor value, Tensor? attn_mask=None, float dropout_p=0., bool is_causal=False, *, float? scale=None, bool enable_gqa=False) -> Tensor

In [61]:
torch_attn()

RuntimeError: aten::_scaled_dot_product_flash_attention_for_cpu() is missing value for argument 'query'. Declaration: aten::_scaled_dot_product_flash_attention_for_cpu(Tensor query, Tensor key, Tensor value, float dropout_p=0., bool is_causal=False, *, Tensor? attn_mask=None, float? scale=None) -> (Tensor output, Tensor logsumexp)